# Imports

In [ ]:
import keras
import tensorflow as tf
import pandas as pd
import os
import collections

import json
import io

import numpy as np

import requests  # Import the requests library

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Barchart

In [ ]:
# File path
file_path = '/content/drive/MyDrive/Colab/DataViz/Barchart/barchart.csv'
df = pd.read_csv(file_path)

# Splitting the 'freq,ind_type,unit,indic_is,geo\TIME_PERIOD' column into separate columns
split_columns = df['freq,ind_type,unit,indic_is,geo\TIME_PERIOD'].str.split(',', expand=True)

# Renaming the columns as per the user's request
split_columns.columns = ['freq', 'ind_type', 'unit', 'indic_is', 'geo']

# Joining the new columns with the original dataframe
df_split = df.join(split_columns)

# Dropping the original combined column
df_split = df_split.drop('freq,ind_type,unit,indic_is,geo\TIME_PERIOD', axis=1)

# Displaying the modified dataframe
df_split.head()

# Remove letters from the dataset
# List of year columns to clean
year_columns = [str(year) for year in range(2012, 2024)]

# Removing leading/trailing spaces from column names
df_split.columns = df_split.columns.str.strip()

# Converting year columns to strings and then removing any letters
for col in year_columns:
    df_split[col] = df_split[col].astype(str).replace(to_replace=r'[a-zA-Z]', value='', regex=True).str.strip()

print(df_split.head())

# Saving the new CSV file
new_csv_path = '/content/drive/MyDrive/Colab/DataViz/Barchart/barchart_processed.csv'
df_split.to_csv(new_csv_path, index=False)

# 2. Linechart

In [ ]:
# File path
file_path = '/content/drive/MyDrive/Colab/DataViz/Linechart/linechart_original.csv'
df = pd.read_csv(file_path)

# Splitting the 'freq,ind_type,unit,indic_is,geo\TIME_PERIOD' column into separate columns
split_columns = df['freq,indic_is,unit,hhtyp,geo\TIME_PERIOD'].str.split(',', expand=True)

# Renaming the columns as per the user's request
split_columns.columns = ['freq', 'ind_type', 'unit', 'indic_is', 'geo']

# Joining the new columns with the original dataframe
df_split = df.join(split_columns)

# Dropping the original combined column
df_split = df_split.drop('freq,indic_is,unit,hhtyp,geo\TIME_PERIOD', axis=1)

# Displaying the modified dataframe
df_split.head()

# Saving the new CSV file
new_csv_path = '/content/drive/MyDrive/Colab/DataViz/Linechart/linechart_processed.csv'
df_split.to_csv(new_csv_path, index=False)

# 3. Treemap

In [ ]:
# File path
file_path = '/content/drive/MyDrive/Colab/DataViz/Treemap/treemap_no_2009.csv'
data = pd.read_csv(file_path)

# Replace ':' with NaN
data = data.replace(r"\s*:\s*", np.nan, regex=True)

years_columns = [str(year) for year in range(2010, 2020)]

# Excluding '2018' from the years as it is not present in the dataset
adjusted_years_columns = [year + " " for year in years_columns if year != '2018']

# Convert year columns to numeric, errors='coerce' will convert non-numeric values to NaN
data[adjusted_years_columns] = data[adjusted_years_columns].apply(pd.to_numeric, errors='coerce')


In [ ]:
# Splitting the 'freq,ind_type,unit,indic_is,geo\TIME_PERIOD' column into separate columns
split_columns = data['freq,indic_is,unit,hhtyp,geo\TIME_PERIOD'].str.split(',', expand=True)

# Renaming the columns as per the user's request
split_columns.columns = ['freq', 'indic_is', 'unit', 'hhtyp', 'geo']

# Joining the new columns with the original dataframe
df_split = data.join(split_columns)

# Dropping the original combined column
df_split = df_split.drop('freq,indic_is,unit,hhtyp,geo\TIME_PERIOD', axis=1)

# Displaying the modified dataframe
df_split.head()

In [ ]:
df_split = df_split.drop(['freq'], axis=1)
df_split = df_split.drop(['hhtyp'], axis=1)
df_split = df_split.drop(['unit'], axis=1)
df_split

In [ ]:
# Group by 'indic_is' and sum values for each year
percentage_summed = df_split.groupby('geo')[adjusted_years_columns].sum().reset_index()
percentage_summed

In [ ]:
# Replace NaN values with 0 for the purpose of calculation
df_split.fillna(0, inplace=True)

# Extract the percentage columns
percentage_columns = df_split.columns[:-4]

# Convert the percentage columns to numeric values
df_split[percentage_columns] = df_split[percentage_columns].apply(pd.to_numeric, errors='coerce')

# Normalize the percentages to sum up to 100 for each year and country
df_normalized = df_split.copy()

In [ ]:
country_list=df_split['geo'].unique()
df_prova=pd.DataFrame()
for year in df_split.columns[:-2]:
  for count in country_list:
    for indic_is in df_split['indic_is'].unique():
      value =  percentage_summed.loc[percentage_summed['geo'] == count, year].values[0]
      single_per = df_split.loc[(df_split['indic_is'] == indic_is) & (df_split['geo'] == count), year]

      if value > 0 :
        df_normalized.loc[(df_split['indic_is'] == indic_is) & (df_split['geo'] == count), year] = ((single_per  * 100) / value)

In [ ]:
new_csv_path = '/content/drive/MyDrive/Colab/DataViz/Treemap/treemap_normalized.csv'
df_normalized.to_csv(new_csv_path, index=False)

In [ ]:
# File path
file_path = '/content/drive/MyDrive/Colab/DataViz/Treemap/treemap_normalized.csv'
data = pd.read_csv(file_path)

In [ ]:
euro_treemap = data[data['geo'] == 'EU28']

In [ ]:
for year in adjusted_years_columns:
  df_result = euro_treemap[['indic_is', year]].copy()
  # Rename columns to 'name' and 'value'
  df_result = df_result.rename(columns={'indic_is': 'name', year: 'value'})
  # Add a 'parent' column with a common root, e.g., 'Origin'
  df_result['parent'] = 'Origin'
  # Create a new row with name = 'Origin' and the other two columns empty
  new_row = pd.DataFrame({'name': ['Origin'], 'value': [''], 'parent': ['']})
  # Append the new row to df_result
  df_result = new_row.append(df_result, ignore_index=True)
  # Replace NaN values with 0 in df_result
  df_result = df_result.fillna(0)
  df_result2=df_result[df_result['value'] != 0]
  new_csv_path = '/content/drive/MyDrive/Colab/DataViz/Treemap/treemap_average'+year[0:4]+'.csv'
  df_result2.to_csv(new_csv_path, index=False)

# 4. Stacked percent barchart

In [ ]:
# File path
file_path = '/content/drive/MyDrive/Colab/DataViz/StackedPercent/stacked_percent.csv'
df = pd.read_csv(file_path)

# Splitting the 'freq,ind_type,unit,indic_is,geo\TIME_PERIOD' column into separate columns
split_columns = df['freq,indic_is,unit,ind_type,geo\TIME_PERIOD'].str.split(',', expand=True)

# Renaming the columns as per the user's request
split_columns.columns = ['freq', 'indic_is', 'unit', 'ind_type', 'geo']

# Joining the new columns with the original dataframe
df_split = df.join(split_columns)

# Dropping the original combined column
df_split = df_split.drop('freq,indic_is,unit,ind_type,geo\TIME_PERIOD', axis=1)

# Displaying the modified dataframe
df_split.head()

# Remove letters from the dataset
# List of year columns to clean (assuming the dataset includes years 2012 to 2023)
year_columns = [str(year) for year in range(2012, 2024)]

# Removing leading/trailing spaces from column names (if any)
df_split.columns = df_split.columns.str.strip()

# Converting year columns to strings and then removing any letters
for col in year_columns:
    df_split[col] = df_split[col].astype(str).replace(to_replace=r'[a-zA-Z]', value='', regex=True).str.strip()

print(df_split.head())

# Saving the new CSV file
new_csv_path = '/content/drive/MyDrive/Colab/DataViz/StackedPercent/stacked_percent_processed.csv'
df_split.to_csv(new_csv_path, index=False)

# 5. Heatmap

In [ ]:
# File path
file_path = '/content/drive/MyDrive/Colab/DataViz/Heatmap/heatmap_processed_2.csv'
df = pd.read_csv(file_path)

# Remove letters from the dataset
# List of year columns to clean (assuming the dataset includes years 2012 to 2023)
year_columns = [str(year) for year in range(2009, 2024)]

# Removing leading/trailing spaces from column names (if any)
df.columns = df.columns.str.strip()

# Converting year columns to strings and then removing any letters
for col in year_columns:
    df[col] = df[col].astype(str).replace(to_replace=r'[a-zA-Z]', value='', regex=True).str.strip()

df.head()
print(df.columns)

data_filtered = df.drop('freq', axis=1)
data_filtered = data_filtered.drop('unit', axis=1)
data_filtered = data_filtered.drop('ind_type', axis=1)


# Saving the new CSV file
new_csv_path = '/content/drive/MyDrive/Colab/DataViz/Heatmap/heatmap_processed_finale.csv'
data_filtered.to_csv(new_csv_path, index=False)

# 6. Piechart

In [ ]:
# File path
file_path = '/content/drive/MyDrive/Colab/DataViz/Piechart/piechart_start.csv'
data = pd.read_csv(file_path)

# Convert ":" to NaN to facilitate checking for missing values
data.replace(":", float("NaN"), inplace=True)

# Ensure the 2013 and 2023 columns are of a suitable type for comparison (e.g., numeric or NaN)
data['2013'] = pd.to_numeric(data['2013'], errors='coerce')
data['2023'] = pd.to_numeric(data['2023'], errors='coerce')

# Group by 'geo' and filter out groups where either 2013 or 2023 columns have NaN values for any 'indic_is'
filtered_data = data.groupby('geo').filter(lambda x: x['2013'].notna().all() and x['2023'].notna().all())

# Display the first few rows of the filtered dataset to verify the result
filtered_data.head()

# Saving the new CSV file
new_csv_path = '/content/drive/MyDrive/Colab/DataViz/Piechart/piechart_processed.csv'
filtered_data.to_csv(new_csv_path, index=False)

# Count the number of unique 'geo' values in the filtered dataset
unique_geo_count = filtered_data['geo'].nunique()
unique_geo_count


# 7.Bubblemap

In [ ]:
# File path
file_path = '/content/drive/MyDrive/Colab/DataViz/Heatmap/heatmap_processed_finale.csv'
data = pd.read_csv(file_path)
data.head()

# Mapping of two-letter "geo" codes to three-letter codes provided by the user
code_mapping = {
    "AL": "ALB", "AT": "AUT", "BA": "BIH", "BE": "BEL", "BG": "BGR", "CH": "CHE",
    "CY": "CYP", "CZ": "CZE", "DE": "DEU", "DK": "DNK", "EE": "EST", "EL": "GRC",
    "ES": "ESP", "FI": "FIN", "FR": "FRA", "HR": "HRV", "HU": "HUN", "IE": "IRL",
    "IS": "ISL", "IT": "ITA", "LT": "LTU", "LU": "LUX", "LV": "LVA", "MK": "MKD",
    "MT": "MLT", "NL": "NLD", "NO": "NOR", "PL": "POL", "PT": "PRT", "RO": "ROU",
    "RS": "SRB", "SE": "SWE", "SI": "SVN", "SK": "SVK", "TR": "TUR", "GB": "GBR"
}

# Map the "geo" column to a new "code" column using the provided mapping
data['code'] = data['geo'].map(code_mapping)

# Display the updated DataFrame to verify the new "code" column
data.head()

# Saving the new CSV file
new_csv_path = '/content/drive/MyDrive/Colab/DataViz/Bubblemap/bubblemap_processed_finale.csv'
data.to_csv(new_csv_path, index=False)


# 8. Small multiple barchart

In [ ]:
# File path
file_path = '/content/drive/MyDrive/Colab/DataViz/SmallMultiple/smallmultiple_original.csv'
df = pd.read_csv(file_path)

# Splitting the 'freq,ind_type,unit,indic_is,geo\TIME_PERIOD' column into separate columns
split_columns = df['freq,ind_type,indic_is,unit,geo\TIME_PERIOD'].str.split(',', expand=True)

# Renaming the columns as per the user's request
split_columns.columns = ['freq', 'ind_type', 'indic_is', 'unit', 'geo']

# Joining the new columns with the original dataframe
df_split = df.join(split_columns)

# Dropping the original combined column
df_split = df_split.drop('freq,ind_type,indic_is,unit,geo\TIME_PERIOD', axis=1)

# Displaying the modified dataframe
df_split.head()

# Saving the new CSV file
new_csv_path = '/content/drive/MyDrive/Colab/DataViz/SmallMultiple/smallmultiple_processed.csv'
df_split.to_csv(new_csv_path, index=False)


file_path = '/content/drive/MyDrive/Colab/DataViz/SmallMultiple/smallmultiple_processed.csv'
df = pd.read_csv(file_path)

year_columns = ["2016", "2018", "2021", "2023"]
# Removing leading/trailing spaces from column names (if any)
df_split.columns = df_split.columns.str.strip()

# Converting year columns to strings and then removing any letters
for col in year_columns:
    df_split[col] = df_split[col].astype(str).replace(to_replace=r'[a-zA-Z]', value='', regex=True).str.strip()

print(df_split.head())

new_csv_path = '/content/drive/MyDrive/Colab/DataViz/StackedPercent/smallmultiple_noletters.csv'
df_split.to_csv(new_csv_path, index=False)

# 9. Ridgeline

In [ ]:
# File path
file_path = '/content/drive/MyDrive/Colab/DataViz/Ridgeline/ridgeline_original.csv'
df = pd.read_csv(file_path)

# Splitting the 'freq,ind_type,unit,indic_is,geo\TIME_PERIOD' column into separate columns
split_columns = df['freq,indic_is,unit,ind_type,geo\TIME_PERIOD'].str.split(',', expand=True)

# Renaming the columns as per the user's request
split_columns.columns = ['freq', 'indic_is', 'unit', 'ind_type', 'geo']

# Joining the new columns with the original dataframe
df_split = df.join(split_columns)

# Dropping the original combined column
df_split = df_split.drop('freq,indic_is,unit,ind_type,geo\TIME_PERIOD', axis=1)

# Displaying the modified dataframe
df_split.head()

# Remove letters from the dataset
# List of year columns to clean (assuming the dataset includes years 2012 to 2023)
year_columns = [str(year) for year in range(2011, 2020)]

# Removing leading/trailing spaces from column names (if any)
df_split.columns = df_split.columns.str.strip()

# Converting year columns to strings and then removing any letters
for col in year_columns:
    df_split[col] = df_split[col].astype(str).replace(to_replace=r'[a-zA-Z]', value='', regex=True).str.strip()

print(df_split.head())

df_split = df_split[df_split['unit'] == 'PC_IND']

# Saving the new CSV file
new_csv_path = '/content/drive/MyDrive/Colab/DataViz/Ridgeline/ridgeline_processed.csv'
df_split.to_csv(new_csv_path, index=False)

# 10. Choropleth Map

In [ ]:
# File path
file_path = '/content/drive/MyDrive/Colab/DataViz/Map/map_original.csv'
df = pd.read_csv(file_path)

# Splitting the 'freq,ind_type,unit,indic_is,geo\TIME_PERIOD' column into separate columns
split_columns = df['freq,indic_is,ind_type,unit,geo\TIME_PERIOD'].str.split(',', expand=True)

# Renaming the columns as per the user's request
split_columns.columns = ['freq', 'indic_is', 'ind_type', 'unit', 'geo']

# Joining the new columns with the original dataframe
df_split = df.join(split_columns)

# Dropping the original combined column
df_split = df_split.drop('freq,indic_is,ind_type,unit,geo\TIME_PERIOD', axis=1)

# Displaying the modified dataframe
df_split.head()

# Remove letters from the dataset
# List of year columns to clean (assuming the dataset includes years 2012 to 2023)
year_columns = ["2015", "2016", "2017", "2019"]

# Removing leading/trailing spaces from column names (if any)
df_split.columns = df_split.columns.str.strip()

# Converting year columns to strings and then removing any letters
for col in year_columns:
    df_split[col] = df_split[col].astype(str).replace(to_replace=r'[a-zA-Z]', value='', regex=True).str.strip()

print(df_split.head())


In [ ]:
countryMapping = {
    "AL": "Albania",
    "AT": "Austria",
    "BA": "Bosnia and Herzegovina",
    "BE": "Belgium",
    "BG": "Bulgaria",
    "CH": "Switzerland",
    "CY": "Cyprus",
    "CZ": "Czech Republic",
    "DE": "Germany",
    "DK": "Denmark",
    "EE": "Estonia",
    "EL": "Greece",
    "ES": "Spain",
    "EU27_2020": "European Union",
    "FI": "Finland",
    "FR": "France",
    "HR": "Croatia",
    "HU": "Hungary",
    "IE": "Ireland",
    "IS": "Iceland",
    "IT": "Italy",
    "LT": "Lithuania",
    "LU": "Luxembourg",
    "LV": "Latvia",
    "MK": "North Macedonia",
    "MT": "Malta",
    "NL": "Netherlands",
    "NO": "Norway",
    "PL": "Poland",
    "PT": "Portugal",
    "RO": "Romania",
    "RS": "Serbia",
    "SE": "Sweden",
    "SI": "Slovenia",
    "SK": "Slovakia",
    "TR": "Turkey",
    "UK": "United Kingdom"
}

# Create a dataframe from the dictionary
df_country_mapping = pd.DataFrame(list(countryMapping.items()), columns=['geo', 'Country'])

# Display the dataframe
print(df_country_mapping)

In [ ]:
# Merge the dataframes using the 'code' and 'geo' columns
merged_df = pd.merge(df_country_mapping, df_split, how='inner')

In [ ]:
dataset2015 = pd.DataFrame()
dataset2015['name'] = merged_df2['Country']
dataset2015['code'] = merged_df2['code']
dataset2015['pop'] = merged_df2['2015']
# Replace ':' with NaN
dataset2015 = dataset2015.replace(r"\s*:\s*", np.nan, regex=True)

dataset2016 = pd.DataFrame()
dataset2016['name'] = merged_df2['Country']
dataset2016['code'] = merged_df2['code']
dataset2016['pop'] = merged_df2['2016']
# Replace ':' with NaN
dataset2016 = dataset2016.replace(r"\s*:\s*", np.nan, regex=True)

dataset2017 = pd.DataFrame()
dataset2017['name'] = merged_df2['Country']
dataset2017['code'] = merged_df2['code']
dataset2017['pop'] = merged_df2['2017']
# Replace ':' with NaN
dataset2017 = dataset2017.replace(r"\s*:\s*", np.nan, regex=True)

dataset2019 = pd.DataFrame()
dataset2019['name']= merged_df2['Country']
dataset2019['code']= merged_df2['code']
dataset2019['pop']= merged_df2['2019']
# Replace ':' with NaN
dataset2019 = dataset2019.replace(r"\s*:\s*", np.nan, regex=True)

In [ ]:
# Saving the new CSV file
new_csv_path = '/content/drive/MyDrive/Colab/DataViz/Map/map_processed_2015.csv'
dataset2015.to_csv(new_csv_path, index=False)
# Saving the new CSV file
new_csv_path = '/content/drive/MyDrive/Colab/DataViz/Map/map_processed_2016.csv'
dataset2016.to_csv(new_csv_path, index=False)
# Saving the new CSV file
new_csv_path = '/content/drive/MyDrive/Colab/DataViz/Map/map_processed_2017.csv'
dataset2017.to_csv(new_csv_path, index=False)
# Saving the new CSV file
new_csv_path = '/content/drive/MyDrive/Colab/DataViz/Map/map_processed_2019.csv'
dataset2019.to_csv(new_csv_path, index=False)